In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle


In [33]:
## Loading the trained model, scaler pickle, onehot
model = load_model('model.h5')

with open('scalers.pickle', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encode_geo = pickle.load(f)

with open('encoder.pkl', 'rb') as f:
    Label_encoder_gender = pickle.load(f)

In [34]:
#Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000,
}

In [35]:
### preprocessing of data
#Geography data
# Convert dictionary to DataFrame
input_df = pd.DataFrame([input_data])
geo_encoder = onehot_encode_geo.transform(input_df[['Geography']]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoder_df.head()
#concatenate the encoded columns back to the input_df
input_df = pd.concat([input_df, geo_encoder_df], axis=1)
# Drop the original 'Geography' column
input_df.drop('Geography', axis=1, inplace=True)


In [36]:
#Gender data
input_df['Gender'] = Label_encoder_gender.transform(input_df['Gender'])
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
#Scale the dinpiata
input_scaler = scaler.transform(input_df)
input_scaler

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
## Prediction
prediction = model.predict(input_scaler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step


array([[0.05021925]], dtype=float32)

In [42]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.05021925)

In [43]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")
    

The customer is not likely to churn.
